# MongoDB and PyMongo [Tutorial](http://api.mongodb.com/python/current/tutorial.html)

## Modules

In [12]:
import pymongo
from pymongo import MongoClient
import datetime
import pprint
from bson.objectid import ObjectId

## Creating connection, database and collection

In [7]:
client = MongoClient('mongodb://localhost:27017/')
db = client['test-database']
collection = db['test-collection']

## Documents

Data in MongoDB is represented (and stored) using JSON-style documents. In PyMongo we use dictionaries to represent documents. As an example, the following dictionary might be used to represent a blog post:

In [93]:
post = {"author": "Mike",
         "text": "My first blog post!",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow()}

### Inserting a Document

In [94]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('592e802c89df412d1cebdee8')

When a document is inserted a special key, *"_id"*, is automatically added if the document doesn’t already contain an *"_id"* key.

After inserting the first document, the posts collection has actually been created on the server. We can verify this by listing all of the collections in our database:

In [95]:
db.collection_names(include_system_collections=False)

['posts']

### Getting a Single Document With find_one()

The most basic type of query that can be performed in MongoDB is find_one(). This method returns a single document matching a query (or None if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match.

In [96]:
pprint.pprint(posts.find_one())

{'_id': ObjectId('592e802c89df412d1cebdee8'),
 'author': 'Mike',
 'date': datetime.datetime(2017, 5, 31, 8, 34, 47, 168000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [97]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('592e802c89df412d1cebdee8'),
 'author': 'Mike',
 'date': datetime.datetime(2017, 5, 31, 8, 34, 47, 168000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [98]:
posts.find_one({"author": "Eliot"}) # No result

### Querying By ObjectId

In [99]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('592e802c89df412d1cebdee8'),
 'author': 'Mike',
 'date': datetime.datetime(2017, 5, 31, 8, 34, 47, 168000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


ObjectId is not the same as its string representation:

In [100]:
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # No result

In [101]:
print(post_id)
print(post_id_as_str)

592e802c89df412d1cebdee8
592e802c89df412d1cebdee8


A common task in web applications is to get an ObjectId from the request URL and find the matching document. It’s necessary in this case to **convert the ObjectId from a string** before passing it to *find_one*:

In [78]:
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

### Bulk Inserts

In addition to inserting a single document, we can also perform bulk insert operations, by passing a list as the first argument to *insert_many()*.

In [102]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]
result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('592e886589df412d1cebdee9'), ObjectId('592e886589df412d1cebdeea')]

There are a couple of interesting things to note about this example:
- The result from *insert_many()* now returns two ObjectId instances, one for each inserted document.
- *new_posts[1]* has a different “shape” than the other posts - there is no *"tags"* field and we’ve added a new field, *"title"*. This is what we mean when we say that MongoDB is *schema-free*.

### Querying for More Than One Document

To get more than a single document as the result of a query we use the find() method. find() returns a Cursor instance, which allows us to iterate over all matching documents:

In [103]:
for post in posts.find():
    pprint.pprint(post)

{'_id': ObjectId('592e802c89df412d1cebdee8'),
 'author': 'Mike',
 'date': datetime.datetime(2017, 5, 31, 8, 34, 47, 168000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('592e886589df412d1cebdee9'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('592e886589df412d1cebdeea'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


In [104]:
for post in posts.find({"author": "Mike"}):
    pprint.pprint(post)

{'_id': ObjectId('592e802c89df412d1cebdee8'),
 'author': 'Mike',
 'date': datetime.datetime(2017, 5, 31, 8, 34, 47, 168000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('592e886589df412d1cebdee9'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


### Counting

In [105]:
posts.count()

3

In [106]:
posts.find({"author": "Mike"}).count()

2

### Range Queries

MongoDB supports many different types of [advanced queries](http://www.mongodb.org/display/DOCS/Advanced+Queries). As an example, lets perform a query where we limit results to posts older than a certain date, but also sort the results by author:

In [107]:
d = datetime.datetime(2009, 11, 12, 12)

# Here we use the special "$lt" operator to do a range query, and also call sort() to sort the results by author.
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('592e886589df412d1cebdeea'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('592e886589df412d1cebdee9'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


### Indexing

Adding [indexes](https://docs.mongodb.com/manual/indexes/) can help accelerate certain queries and can also add additional functionality to querying and storing documents. In this example, we’ll demonstrate how to create a [unique index](http://docs.mongodb.org/manual/core/index-unique/) on a key that rejects documents whose value for that key already exists in the index.

First, we’ll need to create the index:

In [108]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)], unique=True)

sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

Notice that we have two indexes now: one is the index on *_id* that MongoDB creates automatically, and the other is the index on *user_id* we just created.

In [109]:
user_profiles = [
    {'user_id': 211, 'name': 'Luke'},
    {'user_id': 212, 'name': 'Ziltoid'}]

result = db.profiles.insert_many(user_profiles)

The index prevents us from inserting a document whose *user_id* is already in the collection:

In [110]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile)  # This is fine.
result = db.profiles.insert_one(duplicate_profile) # This will DuplicateKeyError

DuplicateKeyError: E11000 duplicate key error collection: test-database.profiles index: user_id_1 dup key: { : 212 }

### Deleting documents/collections

In [111]:
# Delete documents
db.posts.delete_many({})

In [112]:
pprint.pprint(posts.find_one())

None


In [113]:
db.collection_names(include_system_collections=False) # at the moment we have 2 collections

['posts', 'profiles']

In [114]:
# Delete collection
db.profiles.drop()

In [115]:
db.collection_names(include_system_collections=False)

['posts']